# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company site',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog / thought leadership',
   'url': 'https://edwarddonner.com/posts/'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


{'links': [{'type': 'enterprise page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Zhihu', 'url': 'https://www.zhihu.com/org/huggingface'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'Endpoints product', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
1 day ago
•
432k
•
818
deepseek-ai/DeepSeek-OCR
Updated
6 days ago
•
1.48M
•
2.28k
meituan-longcat/LongCat-Video
Updated
2 days ago
•
792
•
214
moonshotai/Kimi-Linear-48B-A3B-Instruct
Updated
about 13 hours ago
•
1.84k
•
187
PaddlePaddle/PaddleOCR-VL
Updated
about 6 hours ago
•
24.6k
•
1.18k
Browse 1M+ models
Spaces
Building
on
CPU Upgrade
358
358
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
15.6k
15.6k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
on
Zero


In [15]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\n1 day ago\n•\n432k\n•\n818\ndeepseek-ai/DeepSeek-OCR\nUpdated\n6 days ago\n•\n1.48M\n•\n2.28k\nmeituan-longcat/LongCat-Video\nUpdated\n2 days ago\n•\n792\n•\n214\nmoonshotai/Kimi-Linear-48B-A3B-Instruct\nUpdated\nabout 13 hours ago\n•\n1.84k\n•\n187\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 6 hours ago\n•\n24.6k\n•\n1.18k\nBrowse 1M+ models\nSpaces\nRunning\n

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face: Where AI Gets Its Hug

## Who Are We?  
Imagine a world where machines don’t just compute, but collaborate, innovate, and make your wildest AI dreams come true. That’s Hugging Face — the buzzing AI community building the *future* one model, dataset, and quirky space at a time.

With over **1 million+ models** and **250k+ datasets**, we’re basically the Netflix of machine learning. And guess what? Our AI snacks come in flavors like text, image, audio, video, and even 3D. Yes, we’re that versatile.

## Why Hugging Face?  
- **Community at the Core**: We’re an open hub where ML engineers, scientists, and hobbyists unite to build ethical AI. No cloak-and-dagger stuff here, just pure, open-source goodness.
- **Collaborate Like a Pro**: Host, share, and discover models or datasets instantly. Want to see what’s trending? Our top models get millions of eyeballs (ok, fine, maybe just a few thousand, but that’s *trending* in AI-speak).
- **Show Off Your Genius**: Build your AI portfolio, share your work, and become the superstar of the ML universe.
- **Compute, Accelerated**: For those who want to kick things into overdrive, our paid compute and enterprise solutions are ready to fuel your AI rocket.

## The Culture — Nerdy, Yet Friendly  
We’re not just about code and algorithms. We’re a family of curious minds exploring the edge of tech—with humor, inclusivity, and a dash of “let’s-make-AI-awesome” spirit. Whether you build deep learning models on a Friday night or chat about image recognition over coffee, here’s where innovation and collaboration hug it out.

 

## Customers & Collaborators  
From AI startups aiming to disrupt the world to tech giants like NVIDIA dropping datasets and tools with us — we’re at the heart of the AI revolution. Our toolkit empowers everyone: researchers, developers, enterprises, and even those who just want to see how far AI can go (trust us, far).

## Careers: Ready to Hug the Future?  
If you'd rather build the next breakthrough than binge-watch cat videos, you should check out our [jobs page](https://huggingface.co/jobs). Join a team where your passion for open-source AI meets real impact. From engineering to research and community roles, if you’re ready to code, create, and collaborate, we’re ready for you.

## How To Join The AI Party  
- **Explore:** Dive into our [Model Hub](https://huggingface.co/models)  
- **Create:** Share your creations and get feedback from a supportive community  
- **Collaborate:** Contribute to cutting-edge projects or start your own in [Spaces](https://huggingface.co/spaces)  
- **Accelerate:** Get enterprise-grade solutions for serious ML muscle  

---

### Fun Fact  

Our logo? It’s a smiley face hugging you back — because at Hugging Face, we're all about warm, fuzzy AI vibes.

---

*Hugging Face — Come for the models, stay for the hugs.*  
[Visit Us](https://huggingface.co) | [Join the Community](https://huggingface.co/join) | [Careers](https://huggingface.co/jobs)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face: Where Machines Get Friendly and AI is a Hug-Fest 🤗

---

### Welcome to Hugging Face — The AI Community Building the Future

Picture a bustling, vibrant playground where machine learning engineers, scientists, and AI enthusiasts from all over the globe come to swap ideas, build cool models, share jaw-dropping datasets, and craft next-gen AI applications. That’s Hugging Face: the place where Big Brains meet Big Data—and make magic happen.

### What’s the Big Deal? Model, Datasets & Spaces Galore!

- **1 Million+** Models at your fingertips: Whether you want an OCR model that reads better than your grandma or a video-generating wizard, Hugging Face has got you covered.
- **250,000+** Datasets: Dive into data pools curated by top names like NVIDIA or join community gems and fuel your AI pursuits.
- **400,000+** Applications in Spaces: Launch AI demos, interactive apps, or tools right on the platform with just a few clicks.

Want multi-modal AI? No problem. Text, image, video, audio, even 3D — Hugging Face lets you explore every AI dimension.

### Community Culture: Open, Inclusive & Driven by Curiosity

Hugging Face isn’t just a platform. It’s a global AI carnival where:

- Collaboration is the secret sauce. Share your work publicly, build your ML profile, or bounce ideas off thousands of fellow AI aficionados.
- Ethics matter. Open-source is king, enabling transparent and responsible AI development that benefits *everyone*.
- Learning is continuous. Whether you’re a machine learning wizard or just a curious cat, the community supports your growth every step of the way.

**Pro tip**: Jump into their forums, join teams, and contribute to projects—because AI gets smarter when it’s social.

### For the Pros (and Aspiring Pros): Careers & Growth Opportunities

Dreaming of a career where your code can change the world? Hugging Face offers:

- A playground rich with cutting-edge tech — think world-class Large Language Models (LLMs) and innovative datasets.
- Opportunities with enterprise clients who demand the best AI solutions.
- A culture that values curiosity, continuous learning, and a sprinkle of fun. (“Serious, but we don’t take ourselves too seriously.”)

If you want to help build the AI community of tomorrow, working at Hugging Face might just be your best career move. 😉  

### Enterprise-Ready: Power Up Your Team with Hugging Face

Got a squad that wants to conquer AI faster and cleaner? Hugging Face delivers:

- Advanced platform access for teams and enterprises.
- Scalable paid compute solutions designed to accelerate your ML projects.
- A partner that's invested in an open, ethical AI future (because who wants a robot uprising?).

### The Lingo You’ll Hear

- **Spaces:** Your AI sandbox to build and share apps.
- **Models:** Pre-trained or custom-made brains for AI tasks.
- **Datasets:** Fuel for teaching your models to get smarter.
- **HF Open Source Stack:** The secret sauce powering all this awesomeness.

---

## Why Hugging Face?

Because where else do you get to merge the power of a billion neurons (AI models) with the warmth of a hug? Hugging Face makes AI human, collaborative, and accessible. They say “It takes a village” — well, they built the village *for* AI.

---

## Ready to Join the Hug Fest?

- Explore models, datasets and apps at [huggingface.co](https://huggingface.co)
- Build your portfolio — show off your AI chops to the world.
- Join a community of tens of thousands pushing AI forward, one model at a time.
- Or maybe even snag a job and hug the future *from the inside.*

Hugging Face: Making AI less intimidating and way more friendly — one model (and one joke) at a time.

---

### Contact & Join the Fun

**Website:** https://huggingface.co  
**Community:** From forums to GitHub, the heart of innovation beats here  
**Careers:** Check [huggingface.co/careers](https://huggingface.co/careers) for open roles and join the revolution!

---

*Did we mention their brand colors are a sunny #FFD21E and a zesty #FF9D00? Bright like their AI future!*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>